In [81]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from datetime import datetime
import os
import sys
import csv
import numpy as np

Start here:

https://www.hasznaltauto.hu/talalatilista/PCOG2VCBN3RTADH4RNPGAO2JXNKY7OYILJQBCFCYUBLYHDIZI6YSIBUSNQ2CT6XXUVWKPFOSTRGAYKMSGMSA24XREA73YLBLFE5ARWESKOYMCQJ4GJLEVYOPNBIJ7IAPGRBEWKKA46AZWGO7FPLWAQB7IAB74LA635SDCJYZDDLOA2VDNRGFESBBVY67BHT2KZJGB37K5TO6NFRDH352VT2VFXUFAH6ATF7MYI2FHF7O7JLIYGA52KWQZLPQRKBCKPIGI37YXDTBWOWUPMGHJQ75EAC7OR26AUGKDPUUTFBQVEMJINVTQBBYYDODJU65GEHZN25ZUEMSXP5QQR3AUXRIGMTYPPWXYHDRUTFALKG4K7S5J7RLO553N74VRJJHGL4A7XNAW3UFHVJHBVP7MJ2H6ZKTKSBJRGFI23PPAOM37SUHC5DYNARC7POCIGPOYYCK7T43RQ6QXJH43JGQ7FLN25FAO5FJQ3OLEAOXIGWZVGOLNLCKHNGUUDLLGY6BR3S6KRKQY7PB6UC7NUKA4PBCBCLV2Y2OKVIS473YWN6WGKJ3GXYABM3E3TDI5VZ6QTHCJWOKUIUH5OVHOR33CWBRA5YPGPI7B7MBHNDZFXIBVDQUYQR4RFP4KDJZTDCSFBMU45QV5VVLEF6RLZGPMMW2ZPE6M4L7WLA2F3GRW3UHVF5SFYLXHH5ZH67THTWCBNEV3TOLN7QIU7FP2HRU6F7TMLKTLGRTOWIQFB36RQBYT7YIAMQVW7IOE3X46HO7J6L6E53RE5UKPXVTJVQRINLWEEXJPL7BJVB7MKGW2A6R7ASVFHGEXXLJMXQCUNFAHHDH5MQK4OO4PVJ7O5SE6Q7KMOO5AH5PTIP4RUU6DS2VSFU6EHTUA62CL7X4RNNAJXLND7PUMPRT7QJX7I44TTPX6GD3NPKBTM2PN6I7PAW3F6Y

In [82]:
filename = '../data/raw/cars.csv'
if not os.path.exists(filename):
    colnames = ['run_datetime', 'query_name', 'make', 'model', 'link', 'price', 'fuel', 'year', 'engine_size', 'engine_power', 'mileage']
    with open(filename, 'w') as file:
        writer = csv.writer(file)
        writer.writerow(colnames)

In [83]:
name = 'Ford Focus'
url = 'https://www.hasznaltauto.hu/talalatilista/PCOG2VCBN3RTADH4RNPGAO2JXNKY7OYILJQBCFCYUBLYHDIZI6YSIBUSNQ2CT6XXUVWKPFOSTRGAYKMSGMSA24XREA73YLBLFE5ARWESKOYMCQJ4GJLEVYOPNBIJ7IAPGRBEWKKA46AZWGO7FPLWAQB7IAB74LA635SDCJYZDDLOA2VDNRGFESBBVY67BHT2KZJGB37K5TO6NFRDH352VT2VFXUFAH6ATF7MYI2FHF7O7JLIYGA52KWQZLPQRKBCKPIGI37YXDTBWOWUPMGHJQ75EAC7OR26AUGKDPUUTFBQVEMJINVTQBBYYDODJU65GEHZN25ZUEMSXP5QQR3AUXRIGMTYPPWXYHDRUTFALKG4K7S5J7RLO553N74VRJJHGL4A7XNAW3UFHVJHBVP7MJ2H6ZKTKSBJRGFI23PPAOM37SUHC5DYNARC7POCIGPOYYCK7T43RQ6QXJH43JGQ7FLN25FAO5FJQ3OLEAOXIGWZVGOLNLCKHNGUUDLLGY6BR3S6KRKQY7PB6UC7NUKA4PBCBCLV2Y2OKVIS473YWN6WGKJ3GXYABM3E3TDI5VZ6QTHCJWOKUIUH5OVHOR33CWBRA5YPGPI7B7MBHNDZFXIBVDQUYQR4RFP4KDJZTDCSFBMU45QV5VVLEF6RLZGPMMW2ZPE6M4L7WLA2F3GRW3UHVF5SFYLXHH5ZH67THTWCBNEV3TOLN7QIU7FP2HRU6F7TMLKTLGRTOWIQFB36RQBYT7YIAMQVW7IOE3X46HO7J6L6E53RE5UKPXVTJVQRINLWEEXJPL7BJVB7MKGW2A6R7ASVFHGEXXLJMXQCUNFAHHDH5MQK4OO4PVJ7O5SE6Q7KMOO5AH5PTIP4RUU6DS2VSFU6EHTUA62CL7X4RNNAJXLND7PUMPRT7QJX7I44TTPX6GD3NPKBTM2PN6I7PAW3F6Y'

In [84]:
page_number = 1
next_page = True
while next_page:
    print(page_number)
    page = requests.get(url + '/page'+str(page_number), verify=False)
    assert page.status_code == 200, page.status_code
    page_number += 1

    soup = bs(page.content, "html.parser")
    results = soup.find_all("div", class_=re.compile("^row talalati-sor"))
    now = datetime.now()

    now_day = now.strftime('%Y_%m_%d')

    for r in results:
        title = r.find("h3").find('a')
        link = title.get('href')
        make, model = title.text.split()[:2]
        
        price = r.select("div[class^=pricefield-primary]")[0].text.strip('Ft')
        price = ''.join(price.split())
        try:
            price = int(price)
        except:
            price = np.nan
    
        details = r.find("div", class_='talalatisor-info adatok')
        fuel, year, engine_size, _, engine_power, mileage = details.text.split(',')
        year = year.split('/')[0]
        engine_size = int(''.join(engine_size.strip(" cm³").split()))
        engine_power = int(''.join(engine_power.strip("LE").split()))
        mileage = int(''.join(mileage.strip('km').split()))
        line = [now, name, make, model, link, price, fuel, year, engine_size, engine_power, mileage]
        with open(filename, 'a') as file:
            writer = csv.writer(file)
            writer.writerow(line)

1


/Users/mveres/opt/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hasznaltauto.hu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


2


/Users/mveres/opt/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hasznaltauto.hu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


3


...

20


/Users/mveres/opt/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hasznaltauto.hu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


21


/Users/mveres/opt/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.hasznaltauto.hu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


AssertionError: 404